This note book downloads and then plots SO2 data from Sentinel-5p TropOMI.

The script downloads the near realtime product which is typically available 2-4 hours after acquisition.

Acquisition usually occurs around 01:00 to 03:00 UTC.  The script should be run at 4pm local time so ensure the data from the previous UTC day is available.

Sometimes there is no realtime data due to operational issues beyond our control.

The time interval can be set in cell 4.

Refer to https://sentinelsat.readthedocs.io/en/stable/api.html#quickstart for more information.

Data can be manually/gui downloaded from
https://scihub.copernicus.eu/

Also see for detailed product information
https://sentinels.copernicus.eu/documents/247904/2474726/Sentinel-5P-Level-2-Product-User-Manual-Sulphur-Dioxide

The next cell imports the required modules and sets the directory to download the data to

In [ ]:
"""
Created on Thu Mar 14 09:12:45 2019

@author: craigm

https://scihub.copernicus.eu/

https://pypi.org/project/sentinelsat/
"""
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json
import os
import zipfile
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
import colorcet as cc
from colorspace import sequential_hcl
from colorspace.colorlib import HCL
import numpy as np



In [ ]:
data_dir = '/opt/data/TropOMI/NZ/'

os.chdir(data_dir)

connect to the API

In [ ]:
api = SentinelAPI('s5pguest', 's5pguest', api_url='https://s5phub.copernicus.eu/dhus')

search by polygon, time, and Hub query keywords

date interval can be 

date : tuple of (str or datetime) or str, optional

A time interval filter based on the Sensing Start Time of the products. Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW"). The timestamps can be either a Python datetime or a string in one of the following formats:

yyyyMMdd

yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)

yyyy-MM-ddThh:mm:ssZ

NOW

NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
    
NOW+<n>DAY(S)
    
yyyy-MM-ddThh:mm:ssZ-<n>DAY(S)
    
NOW/DAY (or HOUR, MONTH etc.) - rounds the value to the given unit

Alternatively, an already fully formatted string such as "[NOW-1DAY TO NOW]" can be used as well.


In [ ]:
date_interval = "[NOW-1DAYS TO NOW]"

In [ ]:
footprint = geojson_to_wkt(read_geojson('/opt/data/TropOMI/NorthIsland.geojson'))
products = api.query(area = footprint, area_relation='Contains', date = date_interval, platformname='Sentinel-5', producttype='L2__SO2___', 
                     processinglevel='L2')


download all results from the search

In [ ]:
download = api.download_all(products, directory_path=data_dir)
download

rename the file to *.nc file (ZIP is the wrong extension)

In [ ]:
for key, value in download[0].items():
    file = download[0][key]['path']
print (file)
nc_file = file.split('.')[0] + '.nc'
newfile = os.rename(file, nc_file)
print (nc_file)

Bounding box parameters for the North Island. qa_value is the pixel quality value, data with values below this are removed. ESA recommends qa_value >0.5 but this sometimes misses low emisions from White Island, so 0.35 seems a good limit for NZ.

In [ ]:
nzlatmin = -40
nzlatmax = -35
nzlonmin = 173
nzlonmax = 179
qa_value = 0.5

Load the data and subset to North Island

In [ ]:
tropomi = xr.open_dataset(nc_file, group='PRODUCT')

#subset to North Island, NZ
ds = tropomi.where((tropomi.latitude > nzlatmin) & (tropomi.latitude < nzlatmax) &
                   (tropomi.longitude > nzlonmin) & (tropomi.longitude < nzlonmax) & (tropomi.qa_value > qa_value))

lons = ds.longitude[:][0,:,:]
lats = ds.latitude[:][0,:,:]
so2 = ds.sulfurdioxide_total_vertical_column[0,:,:]

to_DU = 2241.15
so2_du = so2 * to_DU

Set up the colormap

In [ ]:
# Alterantive color map options
#cmap = cc.m_fire_r
#cmap = cm.gist_ncar_r
#
#palette = sequential_hcl(h = [-220,20], c = [0, 90, 0], l = [95, 30], power = [1.5,1.5])
#cmap = palette.cmap()


lev     = [0., 0.01, 0.1, 0.25, 0.50, 0.75, 1.0, 1.25, 1.50, 1.75, 2.0,
           2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5]
H = np.repeat(-220., len(lev))
H[np.where(np.asarray(lev) >= 1.5)] = -100 # Blueish above 2.0 inches
H[np.where(np.asarray(lev) >= 3.)] = 20   # Reddish above 5.0 inches
C = np.power(np.linspace(0, 1, len(lev), dtype = float), 1.5) * 90
L = 95 - np.power(np.linspace(0, 1, len(lev), dtype = float), 1.5) * 65
# Create a HCL color object
cols = HCL(H, C, L)
# Load colors
palette  = cols.colors()
cmap = LinearSegmentedColormap.from_list('tropomi', palette)

Calculate mass of SO2 in box around White Island
For the unit: I looked in the NASA website: “The gas is measured in Dobson Units (DU), the number of molecules in a square centimeter of the atmosphere. If you were to compress all of the sulfur dioxide in a column of the atmosphere into a flat layer at standard temperature and pressure (0 °C and 1013.25 hPa), one Dobson Unit would be 0.01 millimeters thick and would contain 0.0285 grams of SO2 per square meter.”

In [ ]:
pixel_area = 3500*7000 #sqm of the tropomi pixel
latmin, latmax, lonmin, lonmax = -38,-37, 176.5, 178

white_AOI = tropomi.where((tropomi.latitude > latmin) & (tropomi.latitude < latmax) &
                   (tropomi.longitude > lonmin) & (tropomi.longitude < lonmax) & (tropomi.qa_value > qa_value))

white_so2 = white_AOI.sulfurdioxide_total_vertical_column[0,:,:]
white_so2_du = white_so2 * to_DU

pixel_mass = 0.0285 * white_so2_du # mass/m2 - see note above 

pixel_mass_tot = pixel_mass * pixel_area

white_total_SO2_mass = pixel_mass_tot.sum() #g SO2 

print('White total SO2 = %.1f tonnes' %(white_total_SO2_mass/1000/1000))

Next part does the plotting.

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(8,6))
ax = plt.axes(projection=ccrs.PlateCarree(180))
# draw coastlines
ax.add_feature(cfeature.GSHHSFeature('full', [1,2,3,4]))

ax.set_xlim(-7, -1)
ax.set_ylim(-40,-35)
  
# draw meridians and parallels
ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', draw_labels=True,
             xlocs=np.arange(173, 180, 1), ylocs=np.arange(-40,-34,1), alpha=0.5, linestyle='--')
ax.text(-0.1, 0.55, 'latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes)
ax.text(0.5, -0.1, 'longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes)


# Draw the plot
cs = plt.pcolor(lons, lats, so2_du, cmap=cmap, vmin=(0), vmax=4, transform=ccrs.PlateCarree())
plt.colorbar(cs, ax=ax, label='SO$_{2}$ total vertical column [DU]', orientation='horizontal',fraction=0.045)
# Add Ruapehu and White Island Volcano locations
plt.scatter(-2.8193, -37.519529, marker='o', facecolors='None', s=75, color='g', zorder=1000)
plt.scatter(-4.4356, -39.280836, marker='o', facecolors='None', s=75, color='g', zorder=1000)
rect = patches.Rectangle((lonmin-180, latmin), (lonmax-180)-(lonmin-180), latmax-latmin, linewidth=2, edgecolor='b', facecolor='none')
ax.add_patch(rect)

ax.set_title(str(nc_file.split('/')[5].split('_')[8]) + ' UTC : White Island mass SO${_2}$ = %.1f tonnes: ' %(white_total_SO2_mass/1000/1000) + 'pixel quality > %.2f' %qa_value, pad = 20,
            fontsize=10)

#plt.savefig(data_dir + str(ds.time.values[0])[:10] + '_NZ_tropomi_qa' +str(qa_value) + '.png')
plt.savefig('/opt/html/NorthIsland_TropOMI.png', dpi=300)